In [1]:
import pandas as pd

In [2]:
data_path='/kaggle/input/house-rent-prediction-dataset/House_Rent_Dataset.csv'

In [3]:
model_data=pd.read_csv(data_path)

In [4]:
model_data.isnull().sum() #check if there are any missing values---------->here none

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

In [5]:
model_data.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred',
       'Bathroom', 'Point of Contact'],
      dtype='object')

In [6]:
features=['BHK','Size','Floor','Area Type','Area Locality','Furnishing Status','Bathroom']

In [7]:
X=model_data[features]

In [8]:
categorical_columns=['Floor','Area Type','Area Locality','Furnishing Status']  #convert dtype=object to dtype=category for using category codes to convert object to number
for col in categorical_columns:
    X[col]=X[col].astype('category')
X[categorical_columns]=X[categorical_columns].apply(lambda x:x.cat.codes)
X

/tmp/ipykernel_20/3751477186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col]=X[col].astype('category')
/tmp/ipykernel_20/3751477186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col]=X[col].astype('category')
/tmp/ipykernel_20/3751477186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

,BHK,Size,Floor,Area Type,Area Locality,Furnishing Status,Bathroom
0,2,1100,455,2,221,2,2
1,2,800,14,2,1527,1,1
2,2,1000,14,2,1760,1,1
3,2,800,10,2,526,2,1
4,2,850,10,1,1890,2,1
...,...,...,...,...,...,...,...
4741,2,1000,271,1,219,1,2
4742,3,2000,16,2,1214,1,3
4743,3,1750,271,1,724,1,3
4744,3,1500,226,1,590,1,2


In [9]:
y=model_data.Rent  

In [10]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y=train_test_split(X,y,random_state=1)  #split train and test data

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [12]:
def get_mae(max_leaf_nodes,train_X,val_X,train_y,val_y):
    m_data=RandomForestRegressor(max_leaf_nodes=max_leaf_nodes,random_state=1)
    m_data.fit(train_X,train_y)
    val_predictions=m_data.predict(val_X)
    mae=mean_absolute_error(val_y,val_predictions)
    return(mae)

In [13]:
import numpy as np
best_leaf=0
best_mae=500000
nodes=np.arange(2,2000,200)
for max_leaf in nodes:
    possible_mae=get_mae(max_leaf,train_X,val_X,train_y,val_y)
    if(possible_mae<best_mae):
        best_leaf=max_leaf
best_no_of_leaves=best_leaf


In [14]:
print(best_no_of_leaves)


1802


In [15]:
train_to_rf=RandomForestRegressor(max_leaf_nodes=best_no_of_leaves,random_state=1)
train_to_rf.fit(X,y)
val_predicted_from_rf=train_to_rf.predict(val_X)
improved_mae=mean_absolute_error(val_predicted_from_rf,val_y)
improved_mae

6959.590979480891

In [16]:
print(val_y)

230      18000
4500     10500
3277     16000
4045     13000
1561      9500
         ...  
1614      7500
829     130000
788     200000
1247    180000
3070      8000
Name: Rent, Length: 1187, dtype: int64


In [17]:
print(val_predicted_from_rf)

[ 17166.09640523  12156.01909183  14028.45371867 ... 149827.
 185150.           7102.92795393]
